# Gating with User Defined Functions

Here, we will create a simple gating mechanism that uses a wave function as the gating function.

*Installation and Setup*

In [1]:
%%capture
%pip install psyneulink

In [4]:
import psyneulink as pnl
import numpy as np

First, we create a simple user-defined function (UDF) that will be used as the gating function. It expects an input with two elements: frequency and time. The output is a sinusoidal wave based on the frequency and time, with a phase and amplitude that can be controlled.

In [203]:
def my_sinusoidal_fct(input=[0, 0],
                     phase=0,
                     amplitude=1):
    frequency = input[0]
    t = input[1]
    return amplitude * np.sin(2 * np.pi * frequency * t + phase)

In [212]:
my_sinusoidal_udf = pnl.UserDefinedFunction(
    custom_function=my_sinusoidal_fct,
    phase=1,
    amplitude=1,
    params = {
        pnl.MULTIPLICATIVE_PARAM: 'phase',
        pnl.ADDITIVE_PARAM: 'amplitude'
    }
)
# TODO: finish this up after UDF is fixed

KeyboardInterrupt: 

Now, let's create our composition. It will consist of an input layer, an output layer, and a gating mechanism:

First the input layer:

In [182]:
input_layer = pnl.ProcessingMechanism(
    name='Input',
    input_shapes=2,
    function=pnl.Logistic
)

In [183]:
input_layer.execute([0.5, 0.5])  # Test the input layer with some input

array([[0.62245933, 0.62245933]])

For the output layer, we will specify an additional output port that uses the udf earlier

In [184]:
output_layer = pnl.ProcessingMechanism(
    name='Output',
    input_shapes=2,
    output_ports={
        pnl.NAME: 'Output with UDF',
        pnl.VARIABLE: [(pnl.OWNER_VALUE,0), pnl.EXECUTION_COUNT],  # Here, we use the current value of the mechanism and the execution count as input to the UDF (current value -> frequency, execution count -> time)
        pnl.FUNCTION: my_sinusoidal_fct
    }
)

ComponentError: USER DEFINED FUNCTION-9 (owned by Output with UDF): Illegal arguments in constructor (type: UserDefinedFunction):
	'multiplicative_param'
	'additive_param'

Attention, here we use a function for the output port so the `value` of the mechanism and the content of the output port are not the same:

In [154]:
output_layer.execute([0.5, 0.5, 0.5])

array([[0.5, 0.5, 0.5]])

In [155]:
output_layer.output_port.value # <- since we only have one output port, we can access it directly

array([array([0.5, 0.5, 0.5]), array(1)], dtype=object)

Now, let's create a gating mechanism

In [156]:
gating = pnl.GatingMechanism(
   input_shapes=1,
    gating_signals=[
        output_layer.output_port,
    ]
)

Now, we can create the composition:

In [150]:
comp = pnl.Composition(
    pathways=[[input_layer, output_layer], [gating]],
)

NameError: name 'gating' is not defined